# Clone demo: reusing a plot in multiple canvases

Purpose:
- Show that `layer.clone()` creates a new, independent *view* that shares the same `DataSource`.
- Demonstrate that selections and highlights propagate across clones via the GraphBus, while view state (focus, opacity, zoom) remains per-canvas.
- Keep the event loop inside Jupyter using `%gui qt`.

Run the cells top to bottom. When the two windows open, try dragging a selection rectangle on the scatter in either window; both canvases will show the same highlighted subset. The data is jittered by a `QTimer` to show that both clones update from the same `DataSource`.

In [1]:
%gui qt5

In [2]:
# Imports and project wiring
import os, sys
from PyQt5 import QtWidgets, QtCore
import numpy as np

# Assume this notebook is placed at the project root
sys.path.append(os.getcwd())

from DataSource import DataSource
from canvas.Canvas import Canvas
from graphs.ScatterPlot import ScatterPlot
from graphs.LinePlot import LinePlot

In [3]:
# Make a small synthetic state array compatible with your transforms
rng = np.random.default_rng(0)
N = 500
x = rng.uniform(-5, 5, N)
y = rng.uniform(-5, 5, N)
vx = rng.normal(0, 0.06, N)
vy = rng.normal(0, 0.06, N)
ax = np.zeros(N)
ay = np.zeros(N)
state = np.column_stack((x, y, vx, vy, ax, ay))

ds = DataSource(state)

In [4]:
# Start Qt (integrated with Jupyter by %gui qt) and create two canvases
app = QtWidgets.QApplication.instance() or QtWidgets.QApplication([])

c1 = Canvas(name="Canvas A")
c2 = Canvas(name="Canvas B")
c1.resize(700, 450)
c2.resize(700, 450)
c1.show(); c2.show()

In [5]:
# Base scatter in c1, plus a clone in c2
sp = ScatterPlot(ds, name="Positions")
sp.set_transform(lambda d: d[:, 0:2])  # (x, y)

c1.plot(sp)                 # attaches the original
sp_clone = sp.clone()       # explicit clone for the second canvas
c2.plot(sp_clone)           # attaches the clone

[GraphBus] Registering: ScatterPlot
[GraphBus] Registering: ScatterPlot


In [6]:
# Optional: add a LinePlot in both canvases via clone
lp = LinePlot(ds, sample_rate=1000, transform=lambda d: d[:, 5], name="AccelY")
c1.plot(lp)
c2.plot(lp.clone())

[GraphBus] Registering: LinePlot
[GraphBus] Registering: LinePlot


In [7]:
# Drive shared updates from the same DataSource so both clones update
def step():
    data = ds.get().copy()
    # small random jitter within the [-5, 5] box
    data[:, 0] = np.clip(data[:, 0] + rng.normal(0, 0.02, len(data)), -5, 5)
    data[:, 1] = np.clip(data[:, 1] + rng.normal(0, 0.02, len(data)), -5, 5)
    ds.set(data)

timer = QtCore.QTimer()
timer.timeout.connect(step)
timer.start(50)  # 20 Hz

In [8]:
# Programmatic selection preview: highlight first 50 points from the *original* layer
# You can also drag-select with the mouse in either window; highlights propagate via GraphBus.
import numpy as np
sp.select_indices(np.arange(50))